In [ ]:
import os
import json
import numpy as np
import pandas as pd

In [ ]:
Home = os.getcwd()
DataFolder = os.path.join(Home,'Data')
df = pd.read_csv(os.path.join(DataFolder,'MyDataSet.csv'))
df_cols = list(df.columns)
df = df[df.filter(regex='^(?!Unnamed)').columns]
#df = df.drop(columns=[i for i in df_cols if i.startswith('Unnamed:')])

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

from sklearn_nature_inspired_algorithms.model_selection import NatureInspiredSearchCV
from niapy.algorithms import basic, modified

In [ ]:
def Define_Model():
    model = RandomForestClassifier()
    param_grid = {}
    return model, param_grid

In [ ]:
df_cols = list(df.columns)
y = df[df_cols[-1]]
X = df[df_cols[:-1]]
start_val = 0
model, param_grid = Define_Model()

In [ ]:
def SKLEARN_HBA(Kmodel, Kparam_grid):
    lst = []
    for Kpopulation_size in range(10,101,10): #10
        for Kloudness in np.arange(0.5,1.1,.1): #6
            for Kpulse_rate in np.arange(0.5,1.1,.1): #6
                for Kalpha in np.arange(0.3, Kloudness-0.1, .1):
                    for Kgamma in np.arange(0.3, Kpulse_rate-0.1, .1):
                        for Kmin_frequency in range(0,7,1): #7
                            for Kmax_frequency in range(Kmin_frequency+1, Kmin_frequency+3, 1):
                                Algo = modified.hba.HybridBatAlgorithm()
                                Algo.set_parameters(
                                    population_size = Kpopulation_size,
                                    loudness = Kloudness,
                                    pulse_rate = Kpulse_rate,
                                    alpha = Kalpha,
                                    gamma = Kgamma,
                                    min_frequency = Kmin_frequency,
                                    max_frequency = Kmax_frequency
                                )
                                NIS = NatureInspiredSearchCV(
                                    estimator = Kmodel,
                                    param_grid = Kparam_grid,
                                    algorithm=Algo
                                )
                                lst.append(NIS)
    return lst
'''
def NIAPY_HBA():
    lst = []
    for Kpopulation_size in range(10,101,10): #10
        for Kloudness in np.arange(0.5,1.1,.1): #6
            for Kpulse_rate in np.arange(0.5,1.1,.1): #6
                for Kalpha in np.arange(0.3, Kloudness-0.1, .1):
                    for Kgamma in np.arange(0.3, Kpulse_rate-0.1, .1):
                        for Kdifferential_weight in np.arange(0.1,1.1,0.1): #10
                            for Kcrossover_probability in np.arange(0.1,1.1,.1): #10
                                Kmin_frequency = 0 #for Kmin_frequency in range(0,7,1): #7
                                for Kmax_frequency in range(1, 3, 1):    #for Kmax_frequency in range(Kmin_frequency+1, Kmin_frequency+3, 1):

                                    Algo = modified.hba.HybridBatAlgorithm()
                                    Algo.set_parameters(
                                        population_size = Kpopulation_size,
                                        loudness = Kloudness,
                                        pulse_rate = Kpulse_rate,
                                        alpha = Kalpha,
                                        gamma = Kgamma,
                                        differential_weight = Kdifferential_weight,
                                        crossover_probability = Kcrossover_probability,
                                        min_frequency = Kmin_frequency,
                                        max_frequency = Kmax_frequency,
                                        seed = 1
                                    )
                                    lst.append(Algo)
    return lst
'''

In [ ]:
def Save_Often(HBA_Dict,Name):
    ResultFolder = os.path.join(Home, 'Result')
    Name = os.path.join(ResultFolder, Name)
    with open(Name,'w') as kanaishk:
        json.dump(HBA_Dict,kanaishk)

In [ ]:
NIS_lst = SKLEARN_HBA(model, param_grid)
#Algo_lst = NIAPY_HBA()
print(len(NIS_lst))
Best_Score = 0
Best_Para = None 
HBA_Dict = dict()

In [ ]:
for i in range(len(NIS_lst)):
    if(i%10==0 and i != start_val):
        Name = f'Kanaishk Hybrid Bat Dict-{i/10}.json'
        Save_Often(HBA_Dict,Name)
        HBA_Dict = dict()
    Curr_Candidate = NIS_lst[i]
    Curr_Candidate.fit(X,y)
    Curr_Para = NIS_lst[i].get_algorithm.get_parameters()
    Curr_Score = Curr_Candidate.best_score_
    HBA_Dict[str(Curr_Para)] = Curr_Score
    if Curr_Score > Best_Score:
        Best_Para = Curr_Para

In [ ]:
print(Best_Score)
print(Best_Para)

In [ ]:
#Orignal Paper HBA: https://arxiv.org/abs/1303.6310
#Sklearn Dir: https://github.com/timzatko/Sklearn-Nature-Inspired-Algorithms
#NIAPY ALgorithm Class: https://github.com/NiaOrg/NiaPy/blob/master/niapy/algorithms/algorithm.py
#NIAPY Bat Algorithm Class: https://github.com/NiaOrg/NiaPy/blob/master/niapy/algorithms/basic/ba.py
#NIAPY Hybrid Bat Algorithm Class: https://github.com/NiaOrg/NiaPy/blob/master/niapy/algorithms/modified/hba.py
#NIAPY How to test Algorithm: https://github.com/NiaOrg/NiaPy/blob/master/niapy/tests/test_algorithm.py
#NIAPY How to test HBA: https://github.com/NiaOrg/NiaPy/blob/master/niapy/tests/test_hba.py
#NIAPY Task (Answer to Sphere Function): https://github.com/NiaOrg/NiaPy/blob/master/niapy/task.py